<a href="https://colab.research.google.com/github/Anshlulla/Exploring-RAG/blob/main/RAG_Using_Mistral_and_Weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain weaviate-client tiktoken pypdf rapidocr-onnxruntime -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 831.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 

##Steps in RAG
1. Data Ingestion
2. Data Retrieval
3. Data Generation

###Data Ingestion

In [2]:
!pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.8 MB/s eta 0:00:00


In [4]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Weaviate
import weaviate

In [5]:
# fixing unicode issues in colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
from google.colab import userdata
WEAVIATE_API_KEY = userdata.get('WeaviateAPIKey')

WEAVIATE_URL = "https://gklsuadspq7jghyc7kqiq.c0.asia-southeast1.gcp.weaviate.cloud"

client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY)
)

<ipython-input-6-10fe10febad8>:6: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


In [10]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/state_of_the_union.pdf")
document = loader.load()

In [11]:
document[0]

Document(metadata={'source': '/content/state_of_the_union.pdf', 'page': 0}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members \nof Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \nLast year COVID-19 kept us apart. This year we are finally together again. \nTonight, we meet as Democrats Republicans and Independents. But most importantly as\nAmericans. \nWith a duty to one another to the American people to the Constitution. \nAnd with an unwavering resolve that freedom will always triumph over tyranny. \nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free \nworld thinking he could make it bend to his menacing ways. But he badly \nmiscalculated. \nHe thought he could roll into Ukraine and the world would roll over. Instead he met\na wall of strength he never imagined. \nHe met the Ukrainian people. \nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, \nt

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

<ipython-input-12-1ed3bd10d9e8>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###Chunking of the Data
[Chunkviz](https://chunkviz.up.railway.app) -> can be used to visualize and play around with hyperparameters of splitters

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(separators="\n\n",
                                          chunk_size=500,
                                          chunk_overlap=50) # chunk overlap equivalent to huggingface's context_window
text_chunks = splitter.split_documents(document)

In [41]:
len(text_chunks)

93

###VectorDB

In [42]:
vector_db = Weaviate.from_documents(
    document, embeddings, client=client, by_text=False
)

In [43]:
vector_db.similarity_search("summarize the pdf", k=3)[0].page_content

'Let’s come together to protect our communities, restore trust, and hold law \nenforcement accountable. \nThat’s why the Justice Department required body cameras, banned chokeholds, and \nrestricted no-knock warrants for its officers. \nThat’s why the American Rescue Plan provided $350 Billion that cities, states, and \ncounties can use to hire more police and invest in proven strategies like community\nviolence interruption—trusted messengers breaking the cycle of violence and trauma \nand giving young people hope.  \nWe should all agree: The answer is not to Defund the police. The answer is to FUND \nthe police with the resources and training they need to protect our communities. \nI ask Democrats and Republicans alike: Pass my budget and keep our neighborhoods \nsafe.  \nAnd I will keep doing everything in my power to crack down on gun trafficking and \nghost guns you can buy online and make at home—they have no serial numbers and \ncan’t be traced. \nAnd I ask Congress to pass prov

###Retrieval

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for Question Answering Task.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know
Use maximum of ten sentences and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for Question Answering Task.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know\nUse maximum of ten sentences and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
import torch
from google.colab import userdata

huggingface_hub_api_token = userdata.get('HUGGINGFACE_API_TOKEN')

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

huggingface_hub_api_token = userdata.get('HUGGINGFACE_API_TOKEN')

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", use_auth_token=huggingface_hub_api_token)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             cache_dir="./model/",
                                             device_map = "auto",
                                             torch_dtype=torch.float16,
                                             use_auth_token=huggingface_hub_api_token)

pipe = pipeline("text-generation",
                model = model,
                tokenizer = tokenizer,
                max_new_tokens=512,
                temperature=0.4,
                top_p=0.95,
                repetition_penalty=1.15)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [46]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.llms import HuggingFaceHub
from google.colab import userdata

huggingface_hub_api_token = userdata.get('HUGGINGFACE_API_TOKEN')

# Initialize HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":0.4, "max_new_tokens":512, "top_p":0.95, "repetition_penalty":1.15},
    huggingfacehub_api_token=huggingface_hub_api_token
)

retriever = vector_db.as_retriever()
output_parser = StrOutputParser()

# Correctly use the llm object in the chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [47]:
rag_chain.invoke("What did Biden say about Ukraine")

"Human: You are an assistant for Question Answering Task.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know\nUse maximum of ten sentences and keep the answer concise.\nQuestion: What did Biden say about Ukraine\nContext: [Document(metadata={'page': 0, 'source': '/content/state_of_the_union.pdf'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members \\nof Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \\nLast year COVID-19 kept us apart. This year we are finally together again. \\nTonight, we meet as Democrats Republicans and Independents. But most importantly as\\nAmericans. \\nWith a duty to one another to the American people to the Constitution. \\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free \\nworld thinking he c